In [2]:
import numpy as np

import random
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Conv1D, Activation
from keras.optimizers import Adam

def chunk_maker(v):
    patatesler = []
    for i in range(0, (v.shape[0]) // 14 - 1):
        chunk = v[i*14:(i+1)*14]
        patatesler.append(chunk.reshape(1, 14, 1))
    return np.vstack(patatesler)

def reshaper(v):
    l = v.shape[0]
    return v.reshape(l, 1)

def subsample(l, y, n=1):
    start = random.randint(0, l.shape[0]- (n+1))
    subsample = l[start:start+n]
    suby = y[start: start+n]
    init = np.zeros((len(l) - n, 1))
    return np.concatenate([init, subsample], axis=0), np.concatenate([init, suby], axis=0)

def augment(xs, ys, times=5):
    d, ts, f = xs.shape
    aug_xs = []
    aug_ys = []
    for i in range(d):
        for j in range(times-1):
            random.seed(12)
            q = random.randint(0, ts - 1)
            ax, ay = subsample(xs[i], ys[i], n=q)
            aug_xs.append(ax.reshape(1, ts, f))
            aug_ys.append(ay.reshape(1, ts, f))
    aug_xs = np.vstack(aug_xs)
    aug_ys = np.vstack(aug_ys)
    xs = np.concatenate([xs, aug_xs], axis=0)
    ys = np.concatenate([ys, aug_ys], axis=0)
    return xs, ys

def prep_data(fname):
    days = []
    with open(fname) as f:
        for i, line in enumerate(f):
            if i:
                seq = float(line.rstrip().split(',')[-1])
                days.append(seq)

    days = np.array(days[::-1])
    days -= days.min()
    days /= days.max()

    n = days.shape[0]
    tr = days[:-int(0.2*n)]
    te = days[-int(0.2*n):]
    trd = tr[:-1]
    trl = tr[1:]
    ted = te[:-1]
    tel = te[1:]

    return chunk_maker(trd), chunk_maker(trl), chunk_maker(ted), chunk_maker(tel)

def lstm():
    inp = Input(shape=(14, 1))
    x = LSTM(32, return_sequences=True)(inp)
    x = Dense(1, activation='relu')(x)
    return Model(inputs=inp, outputs=x)

if __name__ == '__main__':
    trd, trl, ted, tel = prep_data('julylstmnonzero.csv')

    trd, trl = augment(trd, trl, times=10)
    ted, tel = augment(ted, tel, times=10)

    np.save('trd.npy', trd)
    np.save('trl.npy', trl)
    np.save('ted.npy', ted)
    np.save('tel.npy', tel)

    epochs = 50
    lr = 1e-3
    decay = lr / epochs
    adam = Adam(lr=lr, decay=decay)

    model = lstm()

    model.summary()
    model.compile(loss='mse', optimizer=adam)
    model.fit(trd, trl, batch_size=14, epochs=epochs, validation_data=[ted, tel], shuffle=True)
    model.save_weights("model.h5")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 14, 1)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 14, 32)            4352      
_________________________________________________________________
dense_1 (Dense)              (None, 14, 1)             33        
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________
Train on 3040 samples, validate on 750 samples
Epoch 1/50
3040/3040 [==============================] - 3s 877us/step - loss: 0.0264 - val_loss: 0.0108
Epoch 2/50
3040/3040 [==============================] - 2s 600us/step - loss: 0.0088 - val_loss: 0.0037
Epoch 3/50
3040/3040 [==============================] - 2s 618us/step - loss: 0.0051 - val_loss: 0.0022
Epoch 4/50
3040/3040 [=======================

In [10]:
sazan = model.load_weights("model.h5")

In [14]:
type(sazan)

NoneType

array([[[  1.21644967e-01],
        [  1.08582986e-01],
        [  9.51839211e-02],
        ..., 
        [  1.35465386e-02],
        [  1.00071630e-02],
        [  7.24729280e-03]],

       [[  5.49867273e-03],
        [  1.45367210e-03],
        [  1.89609405e-04],
        ..., 
        [  2.93051869e-02],
        [  3.28024270e-02],
        [  4.55062571e-02]],

       [[  5.12156070e-02],
        [  5.62297223e-02],
        [  5.38069355e-02],
        ..., 
        [  1.11342856e-01],
        [  1.15387857e-01],
        [  1.24320566e-01]],

       ..., 
       [[  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        ..., 
        [  3.35208360e-01],
        [  3.20566300e-01],
        [  3.04238824e-01]],

       [[  0.00000000e+00],
        [  0.00000000e+00],
        [  0.00000000e+00],
        ..., 
        [  3.35208360e-01],
        [  3.20566300e-01],
        [  3.04238824e-01]],

       [[  0.00000000e+00],
        [  0.00000000e+00],
        [  